### 4. Support Vector Machines

Had a little time to explore SVM just a bit

I would like to come back to this and see if I could improve these scores, but it's pretty overfit, and I spent most of my time on the more promising Multinomial Naive Bayes.

Training score: 0.928343949044586

Testing score: 0.6189928884419222

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np
from sklearn.svm import SVC

In [2]:
df = pd.read_json('../data/cleaned_v1.json')

In [3]:
X = df['ingredients']
y = df['cuisine']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y, test_size=0.7)

In [5]:
pipe1 = Pipeline([
    ('tvec', TfidfVectorizer()),
    ('svm', SVC(gamma='scale'))
])

In [11]:
pipe1_params = {
    'tvec__max_features': [3500, 4000],
    'tvec__max_df': [0.9, 0.95],
    'tvec__min_df': [0.01],
    'svm__C': np.linspace(2, 10, 10),
    'svm__kernel': ['rbf']
}

In [12]:
rs1 = RandomizedSearchCV(estimator=pipe1,
                        param_distributions=pipe1_params,
                        cv=5,
                        scoring="accuracy",
                        n_jobs=-1,
                        verbose=1)

In [13]:
rs1.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 10.6min finished


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('tvec', TfidfVectorizer()),
                                             ('svm', SVC())]),
                   n_jobs=-1,
                   param_distributions={'svm__C': array([ 2.        ,  2.88888889,  3.77777778,  4.66666667,  5.55555556,
        6.44444444,  7.33333333,  8.22222222,  9.11111111, 10.        ]),
                                        'svm__kernel': ['rbf'],
                                        'tvec__max_df': [0.9, 0.95],
                                        'tvec__max_features': [3500, 4000],
                                        'tvec__min_df': [0.01]},
                   scoring='accuracy', verbose=1)

In [14]:
rs1.best_score_

0.6165780627181812

In [15]:
rs1.best_params_

{'tvec__min_df': 0.01,
 'tvec__max_features': 4000,
 'tvec__max_df': 0.95,
 'svm__kernel': 'rbf',
 'svm__C': 2.888888888888889}

In [16]:
rs1.score(X_train, y_train), rs1.score(X_test, y_test)

(0.928343949044586, 0.6189928884419222)